# Unit 3: Flat Surface and Object Recognition

One of the most useful perception skills is being able to recognize objects. This allows you to create robots that can, for instance, grasp objects or understand the world around them a little bit better.<br>
There are two main skills to master here:<br>

* **Recognize flat surfaces**: This skill allows it to detect places where objects normally are, like tables and shelves.It's the first step to searching for objects.


* **Recognize objects**: Once you know where to look, you have to be able to recognise different object in the scene and localise where they are from your robot's location.



You will work on this in a world with a table and a coke can on top of it. The robot you will use is the Fetch robot, which you can move around by using the topic **/base_controller/command**, which uses Twist commands. You can also use other systems for moving the torso up and down and the head to look around. But in this object recognition Unit, only the movement of the base is really necessary.

<img src="img/perception_unit3_demo.png"/>

## Table Top Detector

The first step in recognising objects is knowing where these objects are. For this, you are going to use a port of the <a href="http://wg-perception.github.io/object_recognition_core/">tabletop_object_detector</a> package to be able to detect flat surfaces and represent that detections in RViz.

So, as usual, the first step is to create your own object recognition package:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd;cd ../src
catkin_create_pkg my_object_recognition_pkg rospy object_recognition_core

You will have to add dependencies on each new package you need inside.<br>
So you are going to use this launch file:

<p style="background:green;color:white;">**init_table_top.launch**</p>

In [ ]:
<launch>
    
    <arg name="tabletop_ork_file" value="$(find my_object_recognition_pkg)/conf/detection.tabletop_fetch.ros.ork"/>
    
    <node pkg="object_recognition_core"
    type="detection"
    name="tabletop_server_node"
    args="-c $(arg tabletop_ork_file)"
    output="screen">
    </node>

</launch>

<p style="background:green;color:white;">**END init_table_top.launch**</p>

So, as you can see, you are launching a binary called **detection** with a configuration file as the argument. This file, called **detection.tabletop_fetch.ros.ork,** is where all the input sensors and values for the table detection are set. It's a YAML file with a different extension, .ork. So the first thing is to create a <i>conf</i> directory inside your package. Then, create this file:

<p style="background:green;color:white;">**detection.tabletop_fetch.ros.ork**</p>

In [ ]:
source1:
  type: RosKinect
  module: 'object_recognition_ros.io'
  parameters:
    rgb_frame_id: '/head_camera_rgb_optical_frame'
    rgb_image_topic: '/head_camera/rgb/image_raw'
    rgb_camera_info: '/head_camera/rgb/camera_info'
    depth_image_topic: '/head_camera/depth_registered/image_raw'
    depth_camera_info: '/head_camera/depth_registered/camera_info'
    #
    #crop_enabled: True
    #x_min: -0.4
    #x_max: 0.4
    #y_min: -1.0
    #y_max: 0.2
    #z_min: 0.3
    #z_max: 1.5

sink1:
  type: TablePublisher
  module: 'object_recognition_tabletop'
  inputs: [source1]

pipeline1:
  type: TabletopTableDetector
  module: 'object_recognition_tabletop'
  inputs: [source1]
  outputs: [sink1]
  parameters:
    table_detector:
      min_table_size: 4000
      plane_threshold: 0.01
    #clusterer:
    #  table_z_filter_max: 0.35
    #  table_z_filter_min: 0.025


<p style="background:green;color:white;">**detection.tabletop_fetch.ros.ork**</p>

Of all the parameters you have here, the only ones that are relevant most of the time are the following ones:

rgb_frame_id: '/head_camera_rgb_optical_frame'<br>
rgb_image_topic: '/head_camera/rgb/image_raw'<br>
rgb_camera_info: '/head_camera/rgb/camera_info'<br>
depth_image_topic: '/head_camera/depth_registered/image_raw'<br>
depth_camera_info: '/head_camera/depth_registered/camera_info'<br>

This sets the correct image topics as inputs so that the recognition can be made. Once you have it, just execute the launch file.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch my_object_recognition_pkg init_table_top.launch

The output you should get:

In [ ]:
[ INFO] [1500548343.897178230]: Initialized ROS. node_name: /tabletop_server_node                                                                                
[ INFO] [1500548344.396954355, 244.844000000]: System already initialized. node_name: /tabletop_server_node                                                      
[ INFO] [1500548344.407887012, 244.844000000]: Subscribed to topic:/head_camera/rgb/image_raw [queue_size: 1][tcp_nodelay: 0]                                    
[ INFO] [1500548344.413947574, 244.845000000]: Subscribed to topic:/head_camera/depth_registered/image_raw [queue_size: 1][tcp_nodelay: 0]                       
[ INFO] [1500548344.420596160, 244.845000000]: Subscribed to topic:/head_camera/rgb/camera_info [queue_size: 1][tcp_nodelay: 0]                                  
[ INFO] [1500548344.428558453, 244.846000000]: Subscribed to topic:/head_camera/depth_registered/camera_info [queue_size: 1][tcp_nodelay: 0]                     
126 :: 0.5 , 1 , 0.995004 , /base_link , /head_mount_kinect_rgb_optical_frame                                                                                    
[ INFO] [1500548345.434429706, 244.987000000]: publishing to topic:/tabletop/clusters                                                                            
[ INFO] [1500548345.436181607, 244.987000000]: publishing to topic:/table_array  

And now, open the RVIZ and add all of the elements you want to see (like the Camera element or PointCloud2 elements). To visualize the Table detection, you will have to add **OrkTable** element. You select the topic where the table data is published, in this case, **/table_array**. You can then check certain options, like "bounding_box," to have a bounding box around the detection, or the "top" option to see what is being considered as the top of the surface.

Tabletop 1:

<img src="img/perception_unit3_tabletop1.png"/>

Tabletop 2:

<img src="img/perception_unit3_tabletop2.png"/>

OrkTable configuration:

<img src="img/perception_unit3_tabletop3.png"/>

Let's have a look at all of the data extracted from the **/table_array** topic:

In [ ]:
user ~ $ rostopic info /table_array                                   
Type: object_recognition_msgs/TableArray
Publishers:
 * /tabletop_server_node (http://ip-172-31-35-50:55275/)
Subscribers:                                    
 * /rviz_1500548389692878208 (http://ip-172-31-35-50:47121/)
user ~ $ rosmsg show object_recognition_msgs/TableArray                                   
std_msgs/Header header                                    
  uint32 seq                                    
  time stamp                                    
  string frame_id                                   
object_recognition_msgs/Table[] tables                                    
  std_msgs/Header header                                    
    uint32 seq                                    
    time stamp                                    
    string frame_id                                   
  geometry_msgs/Pose pose                                   
    geometry_msgs/Point position                                    
      float64 x                                   
      float64 y                                   
      float64 z                                   
    geometry_msgs/Quaternion orientation                                    
      float64 x                                   
      float64 y                                   
      float64 z                                   
      float64 w                                   
  geometry_msgs/Point[] convex_hull                                   
    float64 x                                   
    float64 y                                   
    float64 z

As you can see, you can pinpoint the location of any surface detected, or even filter the floor, because you know the height of each surface.

More info:<br>
http://wg-perception.github.io/ork_tutorials/
http://wg-perception.github.io/ork_tutorials/tutorial02/tutorial.html<br>
http://wiki.ros.org/tabletop_object_detector

<p style="background:#EE9023;color:white;">**Exercise U3-1**</p>

Create a script that makes the Fetch robot approach the table based on the data from the /table_array. This will allow you to look for a table and reach it after searching for objects.

<p style="background:green;color:white;">**Help Exercise U3-1**</p>

Here you have the code to make the Fetch robot move and look around. Use it as you see fit:

In [ ]:
#!/usr/bin/env python

import numpy
import copy
import actionlib
import rospy
import time
from math import sin, cos
from moveit_python import (MoveGroupInterface,
                           PlanningSceneInterface,
                           PickPlaceInterface)
from moveit_python.geometry import rotate_pose_msg_by_euler_angles

from control_msgs.msg import FollowJointTrajectoryAction, FollowJointTrajectoryGoal
from control_msgs.msg import PointHeadAction, PointHeadGoal
from geometry_msgs.msg import Twist, Pose
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from moveit_msgs.msg import PlaceLocation, MoveItErrorCodes
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from nav_msgs.msg import Odometry

# Move base using navigation stack
class MoveBasePublisher(object):

    def __init__(self):
        self._move_base_publisher = rospy.Publisher('/base_controller/command', Twist, queue_size=1)
        self.odom_subs = rospy.Subscriber("/odom", Odometry, self.odom_callback)
        self.actual_pose = Pose()
    
    def odom_callback(self,msg):
        self.actual_pose = msg.pose.pose

    def move_to(self, twist_object):
        self._move_base_publisher.publish(twist_object)
    
    def stop(self):
        twist_object = Twist()
        self.move_to(twist_object)
        
    def move_forwards_backwards(self,speed_ms,position_x, allowed_error=0.1):
        """
        distance_metres: positive is go forwards, negative go backwards
        """
        rate = rospy.Rate(5)
        move_twist = Twist()
        
        start_x = self.actual_pose.position.x
        direction_speed = numpy.sign(position_x - start_x)
        move_twist.linear.x = direction_speed*speed_ms
        
        
        in_place = False
        while not in_place:
            self.move_to(move_twist)
            x_actual = self.actual_pose.position.x
            print x_actual

            if  abs(position_x - x_actual) <= allowed_error:
                print "Reached position"
                break

            rate.sleep()
        self.stop()


# Move base using navigation stack
class MoveBaseClient(object):

    def __init__(self):
        self.client = actionlib.SimpleActionClient("move_base", MoveBaseAction)
        rospy.loginfo("Waiting for move_base...")
        self.client.wait_for_server()

    def goto(self, x, y, theta, frame="map"):
        move_goal = MoveBaseGoal()
        move_goal.target_pose.pose.position.x = x
        move_goal.target_pose.pose.position.y = y
        move_goal.target_pose.pose.orientation.z = sin(theta/2.0)
        move_goal.target_pose.pose.orientation.w = cos(theta/2.0)
        move_goal.target_pose.header.frame_id = frame
        move_goal.target_pose.header.stamp = rospy.Time.now()

        # TODO wait for things to work
        self.client.send_goal(move_goal)
        self.client.wait_for_result()

# Send a trajectory to controller
class FollowTrajectoryClient(object):

    def __init__(self, name, joint_names):
        self.client = actionlib.SimpleActionClient("%s/follow_joint_trajectory" % name,
                                                   FollowJointTrajectoryAction)
        rospy.loginfo("Waiting for %s..." % name)
        self.client.wait_for_server()
        self.joint_names = joint_names

    def move_to(self, positions, duration=5.0):
        if len(self.joint_names) != len(positions):
            print("Invalid trajectory position")
            return False
        trajectory = JointTrajectory()
        trajectory.joint_names = self.joint_names
        trajectory.points.append(JointTrajectoryPoint())
        trajectory.points[0].positions = positions
        trajectory.points[0].velocities = [0.0 for _ in positions]
        trajectory.points[0].accelerations = [0.0 for _ in positions]
        trajectory.points[0].time_from_start = rospy.Duration(duration)
        follow_goal = FollowJointTrajectoryGoal()
        follow_goal.trajectory = trajectory

        self.client.send_goal(follow_goal)
        self.client.wait_for_result()

# Point the head using controller
class PointHeadClient(object):

    def __init__(self):
        self.client = actionlib.SimpleActionClient("head_controller/point_head", PointHeadAction)
        rospy.loginfo("Waiting for head_controller...")
        self.client.wait_for_server()

    def look_at(self, x, y, z, frame, duration=1.0):
        goal = PointHeadGoal()
        goal.target.header.stamp = rospy.Time.now()
        goal.target.header.frame_id = frame
        goal.target.point.x = x
        goal.target.point.y = y
        goal.target.point.z = z
        goal.min_duration = rospy.Duration(duration)
        self.client.send_goal(goal)
        self.client.wait_for_result()

# Tools for grasping
class GraspingClient(object):

    def __init__(self):
        self.scene = PlanningSceneInterface("base_link")
        self.pickplace = PickPlaceInterface("arm", "gripper", verbose=True)
        self.move_group = MoveGroupInterface("arm", "base_link")

   
    def getGraspableCube(self):
        
        # nothing detected
        return None, None

    def getSupportSurface(self, name):
        for surface in self.support_surfaces:
            if surface.name == name:
                return surface
        return None

    def getPlaceLocation(self):
        pass

    def pick(self, block, grasps):
        success, pick_result = self.pickplace.pick_with_retry(block.name,
                                                              grasps,
                                                              support_name=block.support_surface,
                                                              scene=self.scene)
        self.pick_result = pick_result
        return success

    def place(self, block, pose_stamped):
        places = list()
        l = PlaceLocation()
        l.place_pose.pose = pose_stamped.pose
        l.place_pose.header.frame_id = pose_stamped.header.frame_id

        # copy the posture, approach and retreat from the grasp used
        l.post_place_posture = self.pick_result.grasp.pre_grasp_posture
        l.pre_place_approach = self.pick_result.grasp.pre_grasp_approach
        l.post_place_retreat = self.pick_result.grasp.post_grasp_retreat
        places.append(copy.deepcopy(l))
        # create another several places, rotate each by 90 degrees in yaw direction
        l.place_pose.pose = rotate_pose_msg_by_euler_angles(l.place_pose.pose, 0, 0, 1.57)
        places.append(copy.deepcopy(l))
        l.place_pose.pose = rotate_pose_msg_by_euler_angles(l.place_pose.pose, 0, 0, 1.57)
        places.append(copy.deepcopy(l))
        l.place_pose.pose = rotate_pose_msg_by_euler_angles(l.place_pose.pose, 0, 0, 1.57)
        places.append(copy.deepcopy(l))

        success, place_result = self.pickplace.place_with_retry(block.name,
                                                                places,
                                                                scene=self.scene)
        return success

    def tuck(self):
        joints = ["shoulder_pan_joint", "shoulder_lift_joint", "upperarm_roll_joint",
                  "elbow_flex_joint", "forearm_roll_joint", "wrist_flex_joint", "wrist_roll_joint"]
        pose = [1.32, 1.40, -0.2, 1.72, 0.0, 1.66, 0.0]
        while not rospy.is_shutdown():
            result = self.move_group.moveToJointPosition(joints, pose, 0.02)
            if result.error_code.val == MoveItErrorCodes.SUCCESS:
                return


def demo():
    # Create a node
    rospy.init_node("face_recognition_demo_node")

    # Make sure sim time is working
    while not rospy.Time.now():
        pass

    
    move_base_pub = MoveBasePublisher()
    
    rospy.loginfo("Start Sequence complete")
    
    rospy.loginfo("Move to position...")
    move_base_pub.move_forwards_backwards(speed_ms=0.3,position_x=1.0, allowed_error=0.1)

if __name__ == "__main__":
    demo()


<p style="background:#EE9023;color:white;">**END Exercise U3-1**</p>

## 2D and 3D Object Finder

This is a basic approach to detecting objects, although you can already differentiate between objects and localize them. For this, you will use **find-object-2d** package.<br>

The first step is to get closer to the table to see the objects as closely as possible, for better detection. You can use your recently created script to move the Fetch robot closer to the table.

### 2D Object Finder

Here you have an example of the launch file you would have to make in order to start the basic system:

<p style="background:green;color:white;">**start_find_object_2d.launch**</p>

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>

<launch> 
    <arg name="camera_rgb_topic" default="/head_camera/rgb/image_raw" />
    <node name="find_object_2d_node" pkg="find_object_2d" type="find_object_2d" output="screen">
        <remap from="image" to="$(arg camera_rgb_topic)"/>
    </node>
    
</launch> 

<p style="background:green;color:white;">**END start_find_object_2d.launch**</p>

You only have to set the RGB camera image source and the system is ready to go. In this case, it's **/head_camera/rgb/image_raw**. Once you launch it, you should go to The Graphical Tools and see something similar to this:

<img src="img/perception_unit3_objectrec2dinit.png"/>

Now, it's time to teach the Fetch robot to recognize a coke can. To do this, select the **Edit->AddObjectFromScene**. You can also add previously taken images directly. But, bear in mind that there are some peculiarities. The images appear in this objectrecogniser mirrored, if you compare them with the images from the cameras directly. So be careful with that.

<img src="img/perception_unit3_mirrorimage1.png"/>

So in the Adding Object menu, you just have to follow the steps to select a region of the image that you consider to be the object. Once done, you should be detecting the object. This system compares the image with the saved ones and looks for matches. If it matches in enough points, it considers it the desired object.

Can Picture:

<img src="img/perception_unit3_addobject1.png"/>

Can detected:

<img src="img/perception_unit3_addobject2.png"/>

Fantastic! Now you can add as many objects as you want, or even turn the object around and take images from different points of view. This will make it detect the object all the time, but keep in mind that without the proper filtering, the system will consider them to be different objects.<br>

So, the last step is to save all of the objects added. There are 2 main ways:<br>

* Saving the Objects as images: File-->Save_Objects. This will save all of the images taken in a folder
* Saving the Whole session: File-->Save_Session. This will save a binary with all of the images and settings. This is the most compact way of doing it, although you won't have access to the images of the objects. It depends on your needs

You can also save the object images directly by right clicking on them, but <font style="color:red">BEWARE</font> that if you do so, these images can't be used for later detection because they aren't mirrored, therefore, the recognition won't work. You can see an example of it here. It's the same image, but one is the manually saved version that is not mirrored and isn't detected, while the other one is correctly saved.

<img src="img/perception_unit3_wrongsaveexample.png"/>

So, once you have your session or your images stored, you need to be able to always start an object recognition session with all of that stored data. To do so, you have the following options:

For saved sessions in my_object_recognition_pkg, in the directory saved_pictures2d:

In [ ]:
<launch>
    <arg name="camera_rgb_topic" default="/head_camera/rgb/image_raw" />
	<!-- Nodes -->
	<node name="find_object_2d" pkg="find_object_2d" type="find_object_2d" output="screen">
		<remap from="image" to="$(arg camera_rgb_topic)"/>
		<param name="gui" value="true" type="bool"/>
		<param name="session_path" value="$(find my_object_recognition_pkg)/saved_pictures2d/coke_session.bin" type="str"/>
		<param name="settings_path" value="~/.ros/find_object_2d.ini" type="str"/>
	</node>

</launch>

For saved images in my_object_recognition_pkg, in the directory saved_pictures2d:

In [ ]:
<launch>
    <arg name="camera_rgb_topic" default="/head_camera/rgb/image_raw" />
	<!-- Nodes -->
	<node name="find_object_2d" pkg="find_object_2d" type="find_object_2d" output="screen">
		<remap from="image" to="$(arg camera_rgb_topic)"/>
		<param name="gui" value="true" type="bool"/>
		<param name="objects_path" value="$(find my_object_recognition_pkg)/saved_pictures2d" type="str"/>
		<param name="settings_path" value="~/.ros/find_object_2d.ini" type="str"/>
	</node>

</launch>

More Info:<br>
http://wiki.ros.org/find_object_2d<br>
http://introlab.github.io/find-object/<br>

<p style="background:#EE9023;color:white;">**Exercise U3-2**</p>

Create a launch file that starts the 2D object recognition through a saved session, where you have saved the coke can object.<br>

<p style="background:#EE9023;color:white;">**END Exercise U3-2**</p>

### Move and Spawn objects 

You have to test your object_recognition system with the same object in different positions, or even in movement.<br>
You also need to test it with various objects in the scene to be sure that it doesn't mistake a human head with a coke.<br>
Here you are going to learn how to move objects around in a Gazebo scene, and spawn new ones also.

To make an object move in a scene, there are 2 steps:<br>

* **Make the object movable in the Gazebo**: For that, you will use the move_model.launch from our spawn_robot_tools_pkg.This makes a topic called "name_object/cmd_vel" available on which you can then publish and move the model around.
* **Publish in the correct topic:** To move the object through the keyboard

If you want to know exactly how this works, <i>we highly recommend you do the RobotIgniteAcademy course in TF and the URDF robot creation</i>.

<p style="background:green;color:white;">**make_cokecan_movable.launch**</p>

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>

<launch> 
    <arg name="robot_name" default="coke_can" />

    <include file="$(find spawn_robot_tools_pkg)/launch/move_model.launch">
        <arg name="robot_name" value="$(arg robot_name)" />
    </include>
</launch>

<p style="background:green;color:white;">**END make_cokecan_movable.launch**</p>

<p style="background:green;color:white;">**move_coke_keyboard.launch**</p>

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>

<launch> 
    <arg name="robot_name" default="coke_can" />
    <node name="$(arg robot_name)_twist_keyboard" pkg="spawn_robot_tools_pkg" type="model_twist_keyboard.py" args="$(arg robot_name)" output="screen"/>
</launch> 

<p style="background:green;color:white;">**move_coke_keyboard.launch**</p>

To move other objects in the scene, just change the name of the model.<br>
To know the name of that the model has in the Gazebo, you can ask the gazebo service:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosservice call /gazebo/get_world_properties

So now, how do you spawn new objects in the scene?<br>
There are many ways to do this. Here, you will learn one.<br>
To learn alternative ways, **<i>we highly recommend that you do the RobotIgniteAcademy course in TF and the URDF robot creation</i>**.

This method needs you to have the sdf files of the models on your package and the whole model installed in the .gazebo/models path. Because you are using RobotIgniteAcademy, you already have the needed models installed there. For simplicity, we have granted access to three of them: beer, coke can, and hammer. You really only need the sdf files of each model. The rest will be retrieved from the gazebo path, not your package. You can copy them to your package by:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd object_recogn_tc<br>
cp models -r /home/user/catkin_ws/src/my_object_recognition_pkg/models

Once you have them, just execute the following launch file to spawn the model where you wish, probably in the table location.<br>
For reference, the coke_can is spawned in the center of the table at XYZ = (-2.0,0.0,0.8).<br>
Bear in mind that if there is already an existing model with the same name, you won't be able to spawn it.

<p style="background:green;color:white;">**spawn_coke.launch**</p>

In [ ]:
<launch>

    <arg name="sdf_robot_file" value="$(find my_object_recognition_pkg)/models/coke_can/model.sdf"/>
    <arg name="robot_name" value="coke_can" />
    <arg name="x" default="-2.0" />
    <arg name="y" default="0.2" />
    <arg name="z" default="0.8" />
    <arg name="yaw" default="0.0" />

    <include file="$(find spawn_robot_tools_pkg)/launch/spawn_sdf.launch">
        <arg name="sdf_robot_file" value="$(arg sdf_robot_file)"/>
        <arg name="robot_name" default="$(arg robot_name)" />
        <arg name="x" value="$(arg x)" />
        <arg name="y" value="$(arg y)" />
        <arg name="z" value="$(arg z)" />
        <arg name="yaw" value="$(arg yaw)" />
    </include>
    
</launch>

<p style="background:green;color:white;">**END spawn_coke.launch**</p>

So, in conclusion, to spawn an object, for example the coke_can, and make it movable at the same time, you will have to execute this launch file:

<p style="background:green;color:white;">**spawn_ready_coke.launch**</p>

In [ ]:
<launch>
    <!-- Spawn CokeCan Model -->
    <include file="$(find my_object_recognition_pkg)/launch/spawn_coke.launch" />
    <!-- Make CokeCan Movable -->
    <include file="$(find my_object_recognition_pkg)/launch/make_cokecan_movable.launch" />
</launch>

<p style="background:green;color:white;">**END spawn_ready_coke.launch**</p>

And afterwards, in another terminal, execute the **move_coke_keyboard.launch**.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch my_object_recognition_pkg spawn_ready_coke.launch

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
roslaunch my_object_recognition_pkg move_coke_keyboard.launch

And to delete the object, just RIGHTCLICK on it and delete it.

<p style="background:#EE9023;color:white;">**Exercise U3-3**</p>

Do the following:<br>

* Delete the existing coke model in the scene.
* Spawn a new coke_can on the table.
* Move the coke can around and see how robust the detection system is with only one picture.
* Then, take as many pictures as you need to never lose track of the object. The best method is to pivot the coke can in place until it stops detecting. Then, add it as a new object, and repeat the process until it detects it from 360 degrees.


<p style="background:#EE9023;color:white;">**END Exercise U3-3**</p>

You should, at the end, have a detection scheme similar to this one:

<img src="img/perception_Unit3_coke360.gif"/>

<p style="background:#EE9023;color:white;">**Exercise U3-4**</p>

Now, do do the same as in Exercise U3-3, but with a beer can and a hammer.<br>
For this, you will have to spawn and move them first.<br>
Remmember to save the session at the end so that you don't lose all of your object images.

<p style="background:#EE9023;color:white;">**END Exercise U3-4**</p>

### 3D Object Detection

Now, the only thing missing is having the position of the objects in the 3D space, to be able to grasp them. To know how to grasp an object, please do the <i>Manipulation course in RobotIgniteAcademy</i>.

The only real difference with the 2D detection will be the sensors involved and the fact that the ObjectPoseStamped will be transformed into TFs.

<p style="background:red;color:white;">**Warning**</p>

You have to create another round of session photos in the 3D system because, otherwise, the detections won't work as well as they should. Especially for the TF transformations.

<p style="background:green;color:white;">**start_find_object_3d_session.launch**</p>

In [ ]:
<launch>
		
	<node name="find_object_3d" pkg="find_object_2d" type="find_object_2d" output="screen">
		<param name="gui" value="true" type="bool"/>
		<param name="settings_path" value="~/.ros/find_object_2d.ini" type="str"/>
		<param name="subscribe_depth" value="true" type="bool"/>
		<param name="session_path" value="$(find my_object_recognition_pkg)/saved_pictures2d/coke_session.bin" type="str"/>
		<param name="objects_path" value="" type="str"/>
		<param name="object_prefix" value="object" type="str"/>
		
		<remap from="rgb/image_rect_color" to="/head_camera/rgb/image_raw"/>
		<remap from="depth_registered/image_raw" to="/head_camera/depth_registered/image_raw"/>
		<remap from="depth_registered/camera_info" to="/head_camera/depth_registered/camera_info"/>
	</node>
	
	<!-- Example of tf synchronisation with the objectsStamped message -->
	<node name="tf_example" pkg="find_object_2d" type="tf_example" output="screen">
		<param name="map_frame_id" value="/map" type="string"/>
		<param name="object_prefix" value="object" type="str"/>
	</node>
	
</launch>

<p style="background:green;color:white;">**END start_find_object_3d_session.launch**</p>

Launching this, you should then get the TF of the object detected published. If you have multiple images of the same object, you will get multiple frames of objects. It's up to you to filter them.

You can also see the object detected by executing the following command in another terminal while the prior launch is working:


<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rosrun find_object_2d print_objects_detected

In [ ]:
...
Object 7 detected, Qt corners at (158.168777,220.963058) (293.200804,220.873430) (158.105995,441.616646) (291.453685,443.002172) 
...

<p style="background:#EE9023;color:white;">**Exercise U3-4**</p>

Do the following:<br>

* Launch the 3D object detector in a session with only one image per object. This way, you won't get a clutter of TF frames.
* Move the objects around through a Python script. This way, you will be able to see how the TFs change and how accurate they are in relation to the real position of the objects.

<p style="background:#EE9023;color:white;">**END Exercise U3-4**</p>

You should get something similar to this:

<img src="img/perception_unit3_object_recognitionTF1.png"/>

And with multiple object images per object, something like this:

<img src="img/perception_unit3_multiple_objects_recognitiontf.png"/>

The TFs appearing can be lowered by decreasing the time you consider a TF obsolete. Because most of these TFs are from previous detections that stay there for a while until they are old enough to be considered irrelevant.

<p style="background:#EE9023;color:white;">**EXTRA Exercise U3-5**</p>

Do the following:<br>

* Spawn two objects on the table and make them move on the table.
* Detect one of the objects.
* Track that object and make the Fetch robot track it with the head to keep the object in the center of its view.


<p style="background:#EE9023;color:white;">**END EXTRA Exercise U3-5**</p>

## Congratulations! You now know how to recognise objects and localise them in space. Go on to the next unit to learn about Human Face Detection.

<p style="background:#417FB1;color:white;">**Project**</p>

You can now do the third exercise of the Aibo Project. There you will have to make the Aibo robot recognise its nice bone and go to it.

<p style="background:#417FB1;color:white;">**END Project**</p>